# CSUSB Study Abroad Chatbot

## **1. Introduction**
The CSUSB Study Abroad Chatbot is a Streamlit-based chatbot that provides information related to study abroad opportunities at California State University, San Bernardino (CSUSB). This chatbot was developed by Team 2 for CSE 6550: Software Engineering Concepts.

In this notebook, we will demonstrate how the chatbot uses retrieval-augmented generation (RAG) to answer questions using study abroad resources as the primary data source.

### Features:
- A **cooldown system** to limit excessive queries and prevent overloading the server.
- **Message persistence** using `st.session_state` to retain chat history.
- A **basic interactive UI** that allows users to input questions and receive responses.


## **2. Installation Requirements**
To run this chatbot, install Streamlit:

Other built-in Python modules used:
- `time`: Used for handling **cooldown timers** and response tracking.


## **3. Code Explanation**

### **3.1 Importing Required Libraries**

In [ ]:
import streamlit as st  # Streamlit for web-based chatbot UI
import time  # Time module for cooldown system

- `streamlit` is used to create the **interactive chatbot UI**.
- `time` is used to **track message cooldown periods** and measure response times.

### **3.2 Cooldown System Configuration**

In [ ]:
COOLDOWN_CHECK_PERIOD: float = 60.  # Time window (in seconds) for checking message frequency
MAX_MESSAGES_BEFORE_COOLDOWN: int = 10  # Maximum messages allowed before cooldown activates
COOLDOWN_DURATION: float = 180.  # Cooldown period in seconds (3 minutes)

- Users can send **10 messages per minute** before hitting a cooldown.
- Once the **limit is exceeded**, users must **wait 3 minutes** before sending more messages.

### **3.3 Cooldown Management Function**

In [ ]:
def canAnswer() -> bool:
    currentTimestamp = time.monotonic()  # Get the current timestamp

- `time.monotonic()` ensures that time **only moves forward**, preventing issues with time tracking.
- This function **determines if the chatbot can answer** based on the cooldown status

In [ ]:
if st.session_state["cooldownBeginTimestamp"] is not None:  # Check if cooldown is active
        if currentTimestamp - st.session_state["cooldownBeginTimestamp"] >= COOLDOWN_DURATION:
            st.session_state["cooldownBeginTimestamp"] = None  # Reset cooldown if time has passed
            return True  # Allow user to send a message

- If **cooldown is active**, it checks whether **enough time has passed**.
- If the cooldown period **has expired**, the chatbot resets the cooldown.

In [ ]:
else:
        st.session_state["messageTimes"] = st.session_state["messageTimes"][-MAX_MESSAGES_BEFORE_COOLDOWN:]
        st.session_state["messageTimes"].append(currentTimestamp)  # Track message timestamps

- Stores only the last **10 message timestamps** (removes old ones to manage memory).
- Ensures that **only recent messages are considered** for cooldown checks.

In [ ]:
 if len(st.session_state["messageTimes"]) <= MAX_MESSAGES_BEFORE_COOLDOWN or            st.session_state["messageTimes"][-1] - st.session_state["messageTimes"][-MAX_MESSAGES_BEFORE_COOLDOWN - 1] >= COOLDOWN_CHECK_PERIOD:
            return True  # Allow message

- Checks if the user has sent fewer than **10 messages per minute**.
- If the limit is not reached, **they can continue chatting**.

In [ ]:
else:
            st.session_state["cooldownBeginTimestamp"] = currentTimestamp  # Start cooldown

- **Activates cooldown mode** if the limit is reached.


In [ ]:
remainingTime = COOLDOWN_DURATION + st.session_state["cooldownBeginTimestamp"] - currentTimestamp
    st.write(f"ERROR: You've reached the limit of {MAX_MESSAGES_BEFORE_COOLDOWN} messages. Please try again in {int(remainingTime//60)} minutes.")
    return False  # Prevent further messages


- **Calculates the remaining cooldown time** and **displays an error message** if the user must wait.

### **3.4 Setting Up Streamlit Chat UI**

In [ ]:
st.html("<h1 style='text-align:center; font-size:48px'>CSUSB Travel Abroad Chatbot</h1>")


- Displays the **chatbot title** in large, centered text for better UI.



### **3.5 Initializing Session Variables**

In [ ]:
if "messages" not in st.session_state or not isinstance(st.session_state["messages"], list):
    st.session_state["messages"] = []  # Store chat history
if "cooldownBeginTimestamp" not in st.session_state:
    st.session_state["cooldownBeginTimestamp"] = None  # Track cooldown start time
if "messageTimes" not in st.session_state:
    st.session_state["messageTimes"] = []  # Store message timestamps

- **Ensures session state variables are initialized**:
  - `messages`: Stores **previous chat messages**.
  - `cooldownBeginTimestamp`: Keeps **track of cooldown activation**.
  - `messageTimes`: Stores **timestamps of sent messages**.

### **3.6 Displaying Chat History**

In [ ]:
for message in st.session_state["messages"]:  # Loop through stored messages
    with st.chat_message(message["role"]):  # Display message
        st.markdown(message["content"])

- **Loops through stored messages** and displays them in the chat window.
- **Keeps past conversations visible**.

### **3.7 Handling User Input**

In [ ]:
prompt = st.chat_input("What is your question?")  # Capture user input
if prompt and canAnswer():  # Check cooldown before processing message

- `st.chat_input()` **creates an input box** for the user.
- Calls `canAnswer()` **to check if the user is allowed to send a message**.

In [ ]:
 st.chat_message("human").markdown(prompt)  # Display user input
    st.session_state["messages"].append({"role": "human", "content": prompt})  # Save message

- **Displays the user’s message** in the chat window.
- **Saves the message** to maintain chat history.

### **3.8 Processing AI Response**

In [ ]:
 responseStartTime = time.monotonic()  # Start response timer
    with st.chat_message("ai"):
        response = "[LLM response here]"  # Placeholder for AI-generated text
        responseEndTime = time.monotonic()  # End response timer
        st.markdown(response)  # Display AI response
        st.session_state["messages"].append({"role": "ai", "content": response})  # Save response

- **Tracks response time** for AI-generated messages.
- Uses a **placeholder AI response** (`"[LLM response here]"`) that can be replaced with an **LLM-generated response**.


In [ ]:
if responseEndTime:  # Display response time for tracking
        st.write(f"*(Last response took {responseEndTime - responseStartTime:.4f} seconds)*")

- **Displays the response time** to track chatbot efficiency.


## **4. Conclusion**


✅ **Prevents message spam** with a cooldown system.  
✅ **Saves chat history** for a seamless conversation flow.  
✅ **Provides AI-generated responses** (can be expanded with OpenAI).  
✅ **Tracks response time** for performance analysis.  

In [ ]:
from langchain_groq import ChatGroq
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import time
from typing import Literal, TypeAlias

AnswerTypes: TypeAlias = Literal["yes", "no", "unanswerable"]

COOLDOWN_CHECK_PERIOD = 60.0  # Time window in seconds to count messages for cooldown
MAX_MESSAGES_BEFORE_COOLDOWN = 10  # Maximum allowed messages within the check period before cooldown
COOLDOWN_DURATION = 180.0  # Duration (in seconds) of the cooldown period once the limit is reached
MAX_RESPONSE_TIME = 3.0  # Maximum acceptable response time in seconds before highlighting slow responses
ANSWER_TYPE_MAX_CHARACTERS_TO_CHECK = 30

SYSTEM_PROMPT = """
You are Llama 3, an expert assistant for the Study Abroad program of California State University, San Bernardino (CSUSB).
Your purpose is to help students with all questions related to studying abroad. You provide detailed, accurate, and helpful information about scholarships, visa processes, university applications, living abroad, cultural adaptation, and academic opportunities worldwide.
Rules & Restrictions:
- **Stay on Topic:** Only respond to questions related to studying abroad, scholarships, university admissions, visas, or life as an international student. If a question is unrelated (e.g. politics or unrelated personal advice), politely guide the user back to study abroad topics.
- **No Negative Responses:** While you must remain truthful at all times, also avoid negative opinions, discouragement, or any response that may deter students from studying abroad.
- **Encourage and Inform:** Provide factual, detailed, and encouraging responses to all study abroad inquiries.
- **No Controversial Discussions:** Do not engage in topics outside of studying abroad, including politics, religion, or personal debates.
- You MUST begin every response with either the phrase "Yes", "No", or "I cannot answer that".
"""

ANSWERABLE_QUESTIONS: dict[str, AnswerTypes] = {
    "does csusb offer study abroad programs?": "yes",
    "can i apply for a study abroad program at csusb?": "yes",
    "is toronto a good place for students to live while studying abroad?": "yes",
    "do i need a visa to study at the university of seoul?": "yes",
    "can i study in south korea or taiwan if I only know english?": "yes"
}
CORRECT_ANSWER_KEYWORDS: tuple[str] = ("yes", "indeed", "correct", "right")
UNANSWERABLE_ANSWER_KEYWORDS: tuple[str] = ("i cannot answer", "i cannot help with", "i do not know")

api_key = ""
cooldownBeginTimestamp: float | None = None
messages = []
messageTimes: list = []
eval_data = {"y_true": [], "y_pred": []}

def canAnswer() -> bool:
    global cooldownBeginTimestamp, messageTimes
    """Check if user can send a new message based on cooldown logic."""
    currentTimestamp = time.monotonic()  # Get the current time in seconds
    if cooldownBeginTimestamp is not None:
        if currentTimestamp - cooldownBeginTimestamp >= COOLDOWN_DURATION:
            cooldownBeginTimestamp = None
            return True
    else:
        messageTimes = messageTimes[-MAX_MESSAGES_BEFORE_COOLDOWN:]
        messageTimes.append(currentTimestamp)
        if (
            len(messageTimes) <= MAX_MESSAGES_BEFORE_COOLDOWN
            or messageTimes[-1] - messageTimes[-MAX_MESSAGES_BEFORE_COOLDOWN - 1] >= COOLDOWN_CHECK_PERIOD
        ):
            return True
        else:
            cooldownBeginTimestamp = currentTimestamp

    cooldownMinutes = int(COOLDOWN_CHECK_PERIOD // 60)
    cooldownSeconds = int(COOLDOWN_CHECK_PERIOD) % 60
    remainingTime = COOLDOWN_DURATION + cooldownBeginTimestamp - currentTimestamp
    remainingMinutes = int(remainingTime // 60)
    remainingSeconds = int(remainingTime) % 60
    print(
        f"ERROR: You've reached the limit of {MAX_MESSAGES_BEFORE_COOLDOWN} questions per "
        f"{cooldownMinutes} minute{'s' if cooldownMinutes != 1 else ''} {cooldownSeconds} second{'s' if cooldownSeconds != 1 else ''}. "
        f"Please try again in {remainingMinutes} minute{'s' if remainingMinutes != 1 else ''} "
        f"{remainingSeconds} second{'s' if remainingSeconds != 1 else ''}."
    )
    return False

def apiBox():
    global api_key
    while True:
        api_key = input("Please enter your Groq API key: ")
        if api_key: break
        print("Invalid key provided. ", end="")

def updateEvalData(question: str, givenAnswer: str) -> None:
    global eval_data
    correctAnswerType = ANSWERABLE_QUESTIONS[question.strip().lower()].lower() if question.strip().lower() in ANSWERABLE_QUESTIONS else "unanswerable"
    if any(keyword.lower() in givenAnswer[:ANSWER_TYPE_MAX_CHARACTERS_TO_CHECK].lower() for keyword in CORRECT_ANSWER_KEYWORDS): givenAnswerType = "yes"
    elif any(keyword.lower() in givenAnswer[:ANSWER_TYPE_MAX_CHARACTERS_TO_CHECK].lower() for keyword in UNANSWERABLE_ANSWER_KEYWORDS): givenAnswerType = "unanswerable"
    else: givenAnswerType = "no"

    eval_data["y_true"].append(correctAnswerType != "unanswerable")
    eval_data["y_pred"].append(givenAnswerType != "unanswerable" and (correctAnswerType == "unanswerable" or givenAnswerType == correctAnswerType))

def render_confusion_matrix():
    global eval_data
    y_true = eval_data["y_true"]
    y_pred = eval_data["y_pred"]
    cm = confusion_matrix(y_true, y_pred, labels=[1, 0])
    TP = cm[0, 0]
    FN = cm[0, 1]
    FP = cm[1, 0]
    TN = cm[1, 1]

    accuracy = accuracy_score(y_true, y_pred) if y_true or y_pred else 0
    precision = precision_score(y_true, y_pred, zero_division=0)
    sensitivity = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    specificity = TN / (TN + FP) if TN + FP else 0

    print(f"""
        =================
             +     -
          ------------- 
        + | {TP:>3} | {FN:>3} |
        - | {FP:>3} | {TN:>3} |
          -------------
        Accuracy: {accuracy:.2f}
        Precision: {precision:.2f}
        Sensitivity: {sensitivity:.2f}
        F1 score: {f1:.2f}
        Specificity: {specificity:.2f}
        =================
    """)

def mainPage():
    global api_key, cooldownBeginTimestamp, messages, messageTimes, eval_data
    apiBox()

    ai = ChatGroq(
        model="llama-3.1-8b-instant",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
        api_key=api_key,  # Pass the API key explicitly.
    )

    responseStartTime, responseEndTime = 0.0, 0.0
    
    while True:
        while True:
            prompt = input("What is your question? ")
            if prompt: break
        if not canAnswer(): continue
        messages.append({"role": "human", "content": prompt})
        sentMessages = [("system", SYSTEM_PROMPT)] + [(m["role"], m["content"]) for m in messages]
        responseStartTime = time.monotonic()
        response = ai.invoke(sentMessages)
        responseEndTime = time.monotonic()
        print("Llama 3: ", response.content)
        messages.append({"role": "ai", "content": response.content})
        
        if responseEndTime:
            responseTime = responseEndTime - responseStartTime
            time_label = f"{responseTime:.4f} seconds"
            print(f"(Last response took {time_label})")
        
        updateEvalData(prompt, response.content)
        render_confusion_matrix()

def main():
    mainPage()

if __name__ == "__main__":
    main()